In [1]:
import pandas as pd
import json
import requests
import datetime
from pprint import pprint
from config import census_api_key
from census import Census
import scipy.stats as st

In [2]:
c = Census(
    census_api_key,
    year = 2017
)

In [3]:
census_data = c.acs5.get(
    (
        "NAME",
        "B19013_001E",
        "B01003_001E",
        "B17001_002E",
        "B23025_005E"
    ),
    {'for': 'zip code tabulation area:*'}
)

# Convert to DataFrame
census_df = pd.DataFrame(census_data)

# Column renaming
census_df = census_df.rename(
    columns = {
        "B01003_001E": "Population",
        "B19013_001E": "Household Income",
        "B17001_002E": "Poverty Count",
        "B23025_005E": 'Unemployed',
        "NAME": "Name",
        "zip code tabulation area": "Zipcode"
    }
)

# Add a Poverty Rate column (Poverty Count / Population)
census_df["Poverty Rate"] = 100 * census_df["Poverty Count"].astype(int) / census_df["Population"].astype(int)

# Configure the final DataFrame
census_df = census_df[
    [
        "Zipcode",
        "Population",
        'Unemployed',
        "Household Income",
        "Poverty Count",
        "Poverty Rate",
        
    ]
]

# Display DataFrame length and sample data
print(f"Number of rows in the DataFrame: {len(census_df)}")
census_df.head()

Number of rows in the DataFrame: 33120


,Zipcode,Population,Unemployed,Household Income,Poverty Count,Poverty Rate
0,00601,17599.0,2454.0,11757.0,11282.0,64.105915
1,00602,39209.0,2538.0,16190.0,20428.0,52.100283
2,00603,50135.0,3588.0,16645.0,25176.0,50.216416
3,00606,6304.0,204.0,13387.0,4092.0,64.911168
4,00610,27590.0,1474.0,18741.0,12553.0,45.498369


In [4]:
poverty_df = census_df.sort_values(by='Poverty Count', ascending=False)
poverty_df.head()

,Zipcode,Population,Unemployed,Household Income,Poverty Count,Poverty Rate
30019,90011,108051.0,3954.0,33824.0,38839.0,35.945063
2645,10456,94667.0,5329.0,26724.0,37540.0,39.654790
27256,78521,92975.0,3280.0,28935.0,34978.0,37.620866
56,00725,82822.0,5766.0,20374.0,34629.0,41.811354
2642,10453,82232.0,4555.0,24421.0,33803.0,41.106868


In [5]:
desired_zip_codes = ['44145', '75206', '55792', '98101', '44312', '79735']

filtered_df = poverty_df[poverty_df['Zipcode'].isin(desired_zip_codes)]

filtered_df

,Zipcode,Population,Unemployed,Household Income,Poverty Count,Poverty Rate
25974,75206,38762.0,922.0,63392.0,5467.0,14.104019
14314,44312,31074.0,1165.0,47393.0,3098.0,9.969750
32338,98101,12408.0,419.0,68750.0,2061.0,16.610251
18733,55792,9795.0,261.0,39080.0,2054.0,20.969883
14256,44145,32387.0,628.0,81966.0,1716.0,5.298422
27714,79735,13383.0,194.0,46445.0,1352.0,10.102369


In [6]:
desired_zip_codes = ['98101', '98102', '98103', '98104', '98105', '98106', '98107', '98108', '98109', '98110', '98111', '98112', '98113', '98114', '98115', '98116', '98117', '98118', '98119', '98121', '98122', '98124', '98125', '98126', '98127', '98129', '98131', '98133','98136', '98138', '98139', '98141', '98144', '98145', '98146', '98148', '98155', '98160', '98161', '98165', '98166', '98168', '98170', '98175', '98177', '98178', '98181', '98185', '98188', '98190', '98191', '98194', '98198', '98199']

seattle_df = poverty_df[poverty_df['Zipcode'].isin(desired_zip_codes)]

seattle_df

,Zipcode,Population,Unemployed,Household Income,Poverty Count,Poverty Rate
32342,98105,47128.0,2112.0,56015.0,11346.0,24.074860
32352,98118,46800.0,2028.0,62504.0,9098.0,19.440171
32358,98133,48390.0,1295.0,60409.0,6247.0,12.909692
32355,98122,37270.0,897.0,72018.0,5835.0,15.656024
32345,98108,24134.0,1241.0,55314.0,5562.0,23.046325
32356,98125,40803.0,1298.0,61014.0,5499.0,13.476950
32369,98168,31771.0,1355.0,53992.0,5381.0,16.936829
32349,98115,51523.0,1159.0,100794.0,5081.0,9.861615
32375,98198,37262.0,1114.0,59843.0,4948.0,13.278944
32372,98178,27279.0,647.0,72478.0,4216.0,15.455112


In [7]:
seattle_df = seattle_df.dropna()

In [8]:
numeric_columns_seattle = seattle_df.select_dtypes(include='number')
seattle_stats_data = numeric_columns_seattle.mean()
seattle_std_data = numeric_columns_seattle.std(skipna=True)

seattle_stats = pd.DataFrame(seattle_stats_data).transpose()
seattle_stats.index = ['Seattle']

seattle_std = pd.DataFrame(seattle_std_data).transpose()
seattle_std.index = ['Seattle Std Dev']

seattle_stats = pd.concat([seattle_stats, seattle_std])

seattle_stats

,Population,Unemployed,Household Income,Poverty Count,Poverty Rate
Seattle,28928.090909,858.212121,77262.303030,3516.787879,11.923184
Seattle Std Dev,11010.669928,448.999566,19789.763898,2372.587838,5.520457


In [9]:
desired_zip_codes = ["75001", "75006", "75007", "75019", "75032", "75039", "75041", "75042", "75043", "75050", "75051", "75052", "75061",
          "75062", "75063", "75080", "75081", "75087", "75088", "75089", "75093", "75098", "75104", "75115", "75116", "75126",
          "75134", "75149", "75150", "75159", "75166", "75180", "75181", "75182", "75201", "75202", "75203", "75204", "75205",
          "75206", "75207", "75208", "75209", "75210", "75211", "75212", "75214", "75215", "75216", "75217", "75218", "75219",
          "75220", "75221", "75222", "75223", "75224", "75225", "75226", "75227", "75228", "75229", "75230", "75231", "75232",
          "75233", "75234", "75235", "75236", "75237", "75238", "75240", "75241", "75242", "75243", "75244", "75246", "75247",
          "75248", "75249", "75250", "75251", "75252", "75253", "75254", "75260", "75261", "75262", "75263", "75264", "75265",
          "75266", "75267", "75270", "75275", "75277", "75283", "75284", "75285", "75287", "75301", "75303", "75312", "75313",
          "75315", "75320", "75326", "75336", "75339", "75342", "75354", "75355", "75356", "75357", "75358", "75359", "75360",
          "75367", "75368", "75370", "75371", "75372", "75373", "75374", "75376", "75378", "75379", "75380", "75381", "75382",
          "75389", "75390", "75391", "75392", "75393", "75394", "75395", "75397", "75398"]

dallas_df = poverty_df[poverty_df['Zipcode'].isin(desired_zip_codes)]

dallas_df

,Zipcode,Population,Unemployed,Household Income,Poverty Count,Poverty Rate
25984,75217,85249.0,1358.0,35282.0,26496.0,31.080716
25983,75216,52895.0,1834.0,25763.0,19585.0,37.026184
25979,75211,77600.0,2219.0,38760.0,19107.0,24.622423
26006,75243,67445.0,2432.0,38640.0,17188.0,25.484469
25993,75228,74605.0,2636.0,40306.0,16217.0,21.737149
...,...,...,...,...,...,...
25975,75207,8028.0,8.0,79926.0,150.0,1.868460
25967,75182,6077.0,166.0,137721.0,139.0,2.287313
25960,75166,4057.0,36.0,91136.0,19.0,0.468326
26018,75390,0.0,0.0,-666666666.0,0.0,NaN


In [10]:
dallas_df = dallas_df.dropna()

In [11]:
numeric_columns_dallas = dallas_df.select_dtypes(include='number')
dallas_stats_data = numeric_columns_dallas.mean()
dallas_std_data = numeric_columns_dallas.std(skipna=True)

dallas_stats = pd.DataFrame(dallas_stats_data).transpose()
dallas_stats.index = ['Dallas']

dallas_std = pd.DataFrame(dallas_std_data).transpose()
dallas_std.index = ['Dallas Std Dev']

dallas_stats = pd.concat([dallas_stats, dallas_std])

dallas_stats

,Population,Unemployed,Household Income,Poverty Count,Poverty Rate
Dallas,32953.134146,988.304878,62511.548780,5436.743902,17.122951
Dallas Std Dev,19615.540120,649.245045,28061.532142,4892.123601,11.045396


In [12]:
desired_zip_codes = ["94101", "94102", "94103", "94104", "94105", "94107", "94108", "94109", "94110", "94111", "94112", "94114",
                 "94115", "94116", "94117", "94118", "94119", "94120", "94121", "94122", "94123", "94124", "94125", "94126",
                 "94127", "94129", "94130", "94131", "94132", "94133", "94134", "94140", "94141", "94142", "94146", "94147",
                 "94157", "94159", "94164", "94165", "94166", "94167", "94168", "94169", "94170", "94172", "94188"]

sanfransisco_df = poverty_df[poverty_df['Zipcode'].isin(desired_zip_codes)]

sanfransisco_df

,Zipcode,Population,Unemployed,Household Income,Poverty Count,Poverty Rate
30985,94112,85373.0,3409.0,82692.0,7750.0,9.077811
30994,94124,35492.0,1854.0,55823.0,7529.0,21.213231
30983,94110,73737.0,2728.0,109747.0,7321.0,9.928530
30976,94102,30140.0,1045.0,33552.0,6957.0,23.082283
30982,94109,56587.0,1610.0,79979.0,6718.0,11.871985
30992,94122,62516.0,1570.0,102838.0,5924.0,9.475974
30977,94103,26990.0,762.0,49052.0,5870.0,21.748796
31000,94132,31155.0,1874.0,72970.0,5124.0,16.446798
31002,94134,43074.0,1845.0,71352.0,5048.0,11.719367
30987,94115,35751.0,1128.0,103625.0,4531.0,12.673771


In [13]:
sanfransisco_df = sanfransisco_df.dropna()

In [14]:
numeric_columns_sanfransisco = sanfransisco_df.select_dtypes(include='number')
sanfransisco_stats_data = numeric_columns_sanfransisco.mean()
sanfransisco_std_data = numeric_columns_sanfransisco.std(skipna=True)

sanfransisco_stats = pd.DataFrame(sanfransisco_stats_data).transpose()
sanfransisco_stats.index = ['Sanfransisco']

sanfransisco_std = pd.DataFrame(sanfransisco_std_data).transpose()
sanfransisco_std.index = ['Sanfransisco Std Dev']

sanfransisco_stats = pd.concat([sanfransisco_stats, sanfransisco_std])

sanfransisco_stats

,Population,Unemployed,Household Income,Poverty Count,Poverty Rate
Sanfransisco,32988.153846,1111.076923,99611.653846,3774.115385,13.602770
Sanfransisco Std Dev,21364.040838,823.142949,43221.162092,2428.532352,9.457921


In [15]:
desired_zip_codes = ["44102", "44103", "44104", "44105", "44106", "44107", "44108", "44109", "44110", "44111", "44112", "44113",
             "44114", "44115", "44117", "44118", "44119", "44120", "44121", "44124", "44125", "44126", "44127", "44128", "44129",
             "44130", "44134", "44135", "44137", "44142", "44143", "44144", "44181", "44188", "44190", "44191", "44192" ,"44193",
             "44194", "44195", "44197", "44198", "44199"]

cleveland_df = poverty_df[poverty_df['Zipcode'].isin(desired_zip_codes)]

cleveland_df

,Zipcode,Population,Unemployed,Household Income,Poverty Count,Poverty Rate
14213,44102,45276.0,2527.0,29909.0,16315.0,36.034544
14216,44105,38018.0,3726.0,25057.0,14715.0,38.705350
14220,44109,40810.0,2622.0,30464.0,12831.0,31.440823
14215,44104,20541.0,2613.0,14646.0,11954.0,58.195804
14231,44120,34556.0,2580.0,33936.0,8452.0,24.458849
14219,44108,22048.0,2449.0,24078.0,8299.0,37.640602
14222,44111,41455.0,1978.0,40542.0,8252.0,19.905922
14217,44106,25836.0,1375.0,25892.0,8183.0,31.672860
14223,44112,21788.0,1683.0,22147.0,8182.0,37.552781
14221,44110,19614.0,1879.0,21468.0,7845.0,39.996941


In [16]:
cleveland_df = cleveland_df.dropna()

cleveland_df

,Zipcode,Population,Unemployed,Household Income,Poverty Count,Poverty Rate
14213,44102,45276.0,2527.0,29909.0,16315.0,36.034544
14216,44105,38018.0,3726.0,25057.0,14715.0,38.705350
14220,44109,40810.0,2622.0,30464.0,12831.0,31.440823
14215,44104,20541.0,2613.0,14646.0,11954.0,58.195804
14231,44120,34556.0,2580.0,33936.0,8452.0,24.458849
14219,44108,22048.0,2449.0,24078.0,8299.0,37.640602
14222,44111,41455.0,1978.0,40542.0,8252.0,19.905922
14217,44106,25836.0,1375.0,25892.0,8183.0,31.672860
14223,44112,21788.0,1683.0,22147.0,8182.0,37.552781
14221,44110,19614.0,1879.0,21468.0,7845.0,39.996941


In [17]:
numeric_columns_cleveland = cleveland_df.select_dtypes(include='number')
cleveland_stats_data = numeric_columns_cleveland.mean()
cleveland_std_data = numeric_columns_cleveland.std(skipna=True)

cleveland_stats = pd.DataFrame(cleveland_stats_data).transpose()
cleveland_stats.index = ['Cleveland']

cleveland_std = pd.DataFrame(cleveland_std_data).transpose()
cleveland_std.index = ['Cleveland Std Dev']

cleveland_stats = pd.concat([cleveland_stats, cleveland_std])

cleveland_stats

,Population,Unemployed,Household Income,Poverty Count,Poverty Rate
Cleveland,26683.75000,1473.50000,37463.125000,6166.937500,26.240029
Cleveland Std Dev,12349.39717,790.67562,14503.969828,3766.234681,14.991650


In [18]:
desired_zip_codes = ["59801", "59802", "59803", "59804", "59806", "59807", "59808"]

missoula_df = poverty_df[poverty_df['Zipcode'].isin(desired_zip_codes)]

missoula_df

,Zipcode,Population,Unemployed,Household Income,Poverty Count,Poverty Rate
20342,59801,32067.0,1457.0,36606.0,7821.0,24.389559
20343,59802,19376.0,850.0,41039.0,3619.0,18.677746
20346,59808,19403.0,536.0,56084.0,2316.0,11.936299
20344,59803,17318.0,335.0,68183.0,1336.0,7.714517
20345,59804,8229.0,246.0,60094.0,918.0,11.155669


In [19]:
missoula_df = missoula_df.dropna()

In [20]:
numeric_columns_missoula = missoula_df.select_dtypes(include='number')
missoula_stats_data = numeric_columns_missoula.mean()
missoula_std_data = numeric_columns_missoula.std(skipna=True)

missoula_stats = pd.DataFrame(missoula_stats_data).transpose()
missoula_stats.index = ['Missoula']

missoula_std = pd.DataFrame(missoula_std_data).transpose()
missoula_std.index = ['Missoula Std Dev']

missoula_stats = pd.concat([missoula_stats, missoula_std])

missoula_stats

,Population,Unemployed,Household Income,Poverty Count,Poverty Rate
Missoula,19278.600000,684.800000,52401.200000,3202.000000,14.774758
Missoula Std Dev,8507.434237,490.058874,13232.491553,2783.552496,6.683550


In [21]:
desired_zip_codes = ['55792']

virginia_minnesota_df = poverty_df[poverty_df['Zipcode'].isin(desired_zip_codes)]

# Assuming virginia_minnesota_df is your original DataFrame
numeric_columns = virginia_minnesota_df.select_dtypes(include='number')
virginia_stats_data = numeric_columns.mean()
virginia_std_data = numeric_columns.std(skipna=True)  # Set skipna=True to exclude NaN values

# Create the DataFrame with mean statistics
virginia_stats = pd.DataFrame(virginia_stats_data).transpose()
virginia_stats.index = ['Virginia Minnesota']

# Concatenate the two DataFrames vertically to add the second row
virginia_stats = pd.concat([virginia_stats])

# Display the final DataFrame
virginia_stats

,Population,Unemployed,Household Income,Poverty Count,Poverty Rate
Virginia Minnesota,9795.0,261.0,39080.0,2054.0,20.969883


In [22]:
desired_zip_codes = ["46107", "46201", "46202", "46203", "46204", "46205", "46206", "46207", "46208", "46214", "46214", "46216",
      "46217", "46217", "46218", "46219", "46220", "46221", "46222", "46224", "46224", "46225", "46226", "46226", "46227",
      "46227", "46228", "46229", "46229", "46230", "46231", "46231", "46234", "46234", "46235", "46235", "46236", "46236",
      "46237", "46237", "46239", "46239", "46240", "46240", "46241", "46241", "46242", "46244", "46247", "46247", "46250",
      "46250", "46251", "46253", "46253" ,"46254", "46254", "46256", "46256", "46259", "46259", "46260", "46260", "46268",
      "46268", "46278", "46278", "46282"]

indianapolis_df = poverty_df[poverty_df['Zipcode'].isin(desired_zip_codes)]

indianapolis_df

,Zipcode,Population,Unemployed,Household Income,Poverty Count,Poverty Rate
15064,46222,36508.0,1449.0,30821.0,12518.0,34.288375
15067,46226,45468.0,3523.0,33348.0,12153.0,26.728688
15065,46224,36881.0,1478.0,35384.0,11687.0,31.688403
15053,46203,39250.0,1711.0,35292.0,10809.0,27.538854
15068,46227,57023.0,1630.0,40415.0,10746.0,18.845027
15051,46201,29852.0,2181.0,28783.0,10270.0,34.403055
15060,46218,26757.0,2038.0,23771.0,10086.0,37.694809
15078,46241,31610.0,1585.0,36090.0,9654.0,30.540968
15073,46235,32578.0,2118.0,36328.0,9233.0,28.341212
15080,46254,40259.0,1778.0,43721.0,7609.0,18.900122


In [23]:
indianapolis_df = indianapolis_df.dropna()

In [24]:
numeric_columns_indy = indianapolis_df.select_dtypes(include='number')
indy_stats_data = numeric_columns_indy.mean()
indy_std_data = numeric_columns_indy.std(skipna=True)  # Set skipna=True to exclude NaN values

indy_stats = pd.DataFrame(indy_stats_data).transpose()
indy_stats.index = ['Indianapolis']

indy_std = pd.DataFrame(indy_std_data).transpose()
indy_std.index = ['Indy Std Dev']

indy_stats = pd.concat([indy_stats, indy_std])

indy_stats

,Population,Unemployed,Household Income,Poverty Count,Poverty Rate
Indianapolis,26490.222222,1087.055556,50769.527778,5070.083333,17.588272
Indy Std Dev,12050.867189,729.706279,19763.152710,3954.195250,9.985247


In [25]:
combined_stats = pd.concat([cleveland_stats, dallas_stats, indy_stats, missoula_stats, sanfransisco_stats, seattle_stats, virginia_stats])
combined_stats

,Population,Unemployed,Household Income,Poverty Count,Poverty Rate
Cleveland,26683.750000,1473.500000,37463.125000,6166.937500,26.240029
Cleveland Std Dev,12349.397170,790.675620,14503.969828,3766.234681,14.991650
Dallas,32953.134146,988.304878,62511.548780,5436.743902,17.122951
Dallas Std Dev,19615.540120,649.245045,28061.532142,4892.123601,11.045396
Indianapolis,26490.222222,1087.055556,50769.527778,5070.083333,17.588272
Indy Std Dev,12050.867189,729.706279,19763.152710,3954.195250,9.985247
Missoula,19278.600000,684.800000,52401.200000,3202.000000,14.774758
Missoula Std Dev,8507.434237,490.058874,13232.491553,2783.552496,6.683550
Sanfransisco,32988.153846,1111.076923,99611.653846,3774.115385,13.602770
Sanfransisco Std Dev,21364.040838,823.142949,43221.162092,2428.532352,9.457921
